# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [ ]:
!pip install --upgrade gdown

# Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

!unzip -q libriphone.zip
!ls libriphone

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [ ]:
# 导入必要的库
import numpy as np  # 引入numpy库，用于高级数学计算和数据处理
import torch       # 引入PyTorch库，一个强大的深度学习框架
import random     # 引入random库，用于生成随机数

# 定义一个函数same_seeds，目的是设置所有可能的随机种子以确保实验的可复现性
def same_seeds(seed):
    # 使用random.seed设置Python内置随机模块的种子，确保每次运行时生成的随机序列相同
    random.seed(seed)

    # 使用np.random.seed设置numpy的随机种子，对于numpy相关的随机操作，这能保证结果的确定性
    np.random.seed(seed)

    # 使用torch.manual_seed设置PyTorch的CPU随机种子，这对于基于CPU的张量操作和模型权重初始化等是必要的
    torch.manual_seed(seed)

    # 检查是否支持CUDA（GPU加速）
    if torch.cuda.is_available():
        # 如果支持CUDA，设置当前GPU的随机种子，确保GPU上的操作也是确定性的
        torch.cuda.manual_seed(seed)

        # 为所有GPU设置随机种子，这对于多GPU训练非常重要，确保每个GPU上的操作具有相同的随机性
        torch.cuda.manual_seed_all(seed)

    # 设置cudnn（一个在CUDA上的深度学习加速库）的行为，使其不使用benchmark模式
    # benchmark模式会自动寻找最适合当前硬件配置的最佳卷积算法，关闭它是为了结果的可复现性
    torch.backends.cudnn.benchmark = False

    # 设置cudnn为确定性模式，这意味着在多次执行时，给定相同的输入，将得到完全相同的输出
    # 这对于科研和调试非常重要，因为它确保了实验结果是可以复现的
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

用于从每个话语的原始MFCC特征中预处理训练数据的辅助函数：
一个音素可能跨越多个帧，并且依赖于其前后的帧。因此，为了提高准确性，我们在训练时会连接相邻的音素。concat_feat函数负责拼接过去的k帧和未来的k帧（共计2k+1=n帧），而我们的目标是预测中间的那一帧。
您可以随意修改数据预处理函数，但务必不要遗漏任何帧（如果您对函数进行了修改，请确保检查帧的数量是否与讲义中提到的数量一致）。

MFCC是Mel Frequency Cepstral Coefficients（梅尔频率倒谱系数）的缩写，是一种在语音处理、音频识别和自然语言处理等领域广泛应用的特征提取技术。它基于人耳的听觉感知特性设计，能够有效地捕捉和表示语音信号中的重要信息。


In [ ]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    # 这个函数用于从指定路径加载已经预处理过的特征数据
    # torch.load()可以加载之前使用torch.save()保存的张量、模型等对象
    feat = torch.load(path)
    return feat  # 返回加载的特征数据

def shift(x, n):
    # 此函数对输入的张量x进行平移操作，n决定了向左或向右平移的帧数
    if n < 0:  # 如果n为负，表示向左平移
        left = x[0].repeat(-n, 1)  # 复制第一帧，重复-n次，作为左侧填充
        right = x[:n]              # 取x的前n帧作为右侧部分
    elif n > 0:  # 如果n为正，表示向右平移
        right = x[-1].repeat(n, 1) # 复制最后一帧，重复n次，作为右侧填充
        left = x[n:]               # 取x从第n帧开始的所有帧作为左侧部分
    else:  # 如果n为0，不需要平移，直接返回原张量
        return x

    # 使用torch.cat将左侧和右侧的部分在第0维度（行）拼接起来
    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    # 确保concat_n是奇数，因为我们要确保有一个明确的中心帧
    assert concat_n % 2 == 1, "concat_n必须是奇数"

    # 如果concat_n小于2，没有拼接的意义，直接返回原数据
    if concat_n < 2:
        return x

    # 获取x的序列长度和特征维度
    seq_len, feature_dim = x.size(0), x.size(1)

    # 将x沿着特征维度重复concat_n次，以准备拼接
    x = x.repeat(1, concat_n)

    # 调整形状为[seq_len, concat_n, feature_dim]，然后转置为[concat_n, seq_len, feature_dim]
    # 这样做是为了便于按中心帧进行拼接
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2)

    # 计算中间帧索引
    mid = concat_n // 2

    # 对每一个远离中心的帧，应用shift函数进行平移
    for r_idx in range(1, mid+1):  # 从1到mid（包含）
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)  # 向右平移
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)  # 向左平移

    # 最后，将张量的形状调整回[seq_len, concat_n*feature_dim]，完成特征拼接
    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    """
    预处理数据函数，用于根据数据集划分（训练、验证或测试）加载特征和标签，同时进行数据拼接处理。

    参数:
    - split: 字符串，表示数据集的划分，可以是'train', 'val', 或 'test'。
    - feat_dir: 字符串，特征文件所在的目录路径。
    - phone_path: 字符串，包含数据标签和分割信息的目录路径。
    - concat_nframes: 整数，表示拼接前后多少帧来创建新的特征向量。
    - train_ratio: 浮点数，默认0.8，训练集与验证集划分的比例。
    - random_seed: 整数，默认1213，用于随机数生成器的种子值，确保实验可复现。

    返回:
    - X: 张量，包含预处理后的特征数据。
    - y: 张量（仅训练模式下），包含对应的标签数据。
    """
    # 预定义的类别数，通常根据数据集而定，此处为41类
    class_num = 41

    # 根据split参数决定数据处理模式
    mode = 'train' if split in ['train', 'val'] else 'test'
    if mode not in ['train', 'val', 'test']:
        raise ValueError('无效的\'split\'参数值，应为\'train\'、\'val\'或\'test\'。')

    # 初始化标签字典，用于训练模式下加载标签
    label_dict = {} if mode == 'train' else None

    # 训练模式下加载标签
    if mode == 'train':
        with open(os.path.join(phone_path, f'{mode}_labels.txt'), 'r') as f:
            for line in f:
                line_parts = line.strip().split(' ')
                label_dict[line_parts[0]] = [int(p) for p in line_parts[1:]]

        # 读取训练集分割文件并随机打乱
        with open(os.path.join(phone_path, 'train_split.txt'), 'r') as f:
            usage_list = f.readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        # 根据split参数选择训练集或验证集
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]
    elif mode == 'test':
        # 测试集直接读取分割文件
        with open(os.path.join(phone_path, 'test_split.txt'), 'r') as f:
            usage_list = f.readlines()

    # 清理文件名列表，去除换行符
    usage_list = [line.strip('\n') for line in usage_list]
    # 打印数据集信息
    print(f'[Dataset] - 音素类别数: {class_num}, {split}集中的话语数量: {len(usage_list)}')

    # 初始化特征矩阵X和标签矩阵y（仅训练模式）
    max_len = 3000000  # 预设的最大序列长度
    X = torch.empty(max_len, 39 * concat_nframes)
    y = torch.empty(max_len, dtype=torch.long) if mode == 'train' else None

    # 遍历数据集文件名，加载特征，进行拼接，并存入X中；若为训练模式，则同时处理标签
    idx = 0  # 当前已处理的样本数量
    for i, fname in tqdm(enumerate(usage_list), desc=f"Processing {split} data"):  # 添加进度条提示
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)  # 对特征进行拼接处理
        if mode == 'train':
            label = torch.LongTensor(label_dict[fname])  # 加载对应标签

        # 将当前样本的特征和标签添加到矩阵X和y中
        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
            y[idx: idx + cur_len] = label

        idx += cur_len  # 更新已处理样本计数

    # 调整X的大小，去除未使用的部分
    X = X[:idx, :]
    if mode == 'train':
        y = y[:idx]  # 调整y的大小，去除未使用的部分

    # 打印处理完毕后的数据集信息
    print(f'[INFO] {split}集处理完成')
    print(X.shape)
    if mode == 'train':
        print(y.shape)
        return X, y  # 返回特征和标签
    else:
        return X  # 测试集只返回特征


# Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

# 定义一个名为LibriDataset的类，继承自torch的Dataset类，用于自定义数据集处理
class LibriDataset(Dataset):
    # 初始化方法，用于数据集的设置
    def __init__(self, X, y=None):
        # X代表数据集的特征部分，将其赋值给实例变量self.data
        self.data = X
        # 判断是否有标签数据y
        if y is not None:
            # 若y存在，则将其转换为torch的LongTensor类型，并赋值给实例变量self.label，用于存储标签数据
            self.label = torch.LongTensor(y)
        else:
            # 若y不存在，则将self.label设为None，表示这是一个无标签数据集
            self.label = None

    # __getitem__方法定义了如何获取数据集中的某个元素，idx为索引
    def __getitem__(self, idx):
        # 判断数据集是否有标签
        if self.label is not None:
            # 如果有标签，则返回当前索引idx指向的数据样本及其对应的标签
            return self.data[idx], self.label[idx]
        else:
            # 如果没有标签，则只返回数据样本
            return self.data[idx]

    # __len__方法定义了数据集的长度，即数据样本的数量
    def __len__(self):
        # 直接返回数据样本列表（self.data）的长度
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [ ]:
import torch.nn as nn

# 定义一个基础模块类，这是神经网络的基本构建单元
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        # 继承自nn.Module类，初始化父类
        super(BasicBlock, self).__init__()

        # 创建一个序列容器Sequential，它会按顺序应用其中的模块
        # 当前模块包含一个线性变换层（nn.Linear），用于调整输入到输出的维度
        # 紧接着是一个ReLU激活函数，用于增加网络的非线性
        # 注意：我们可以添加批量归一化（Batch Normalization）和丢弃层（Dropout）
        # 以增强模型的泛化能力和训练稳定性。你可以通过查阅nn.BatchNorm1d和nn.Dropout添加这些层
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),  # 线性变换层
            nn.ReLU(),  # 激活函数，让模型能学习更复杂的模式
        )

    # 前向传播方法，定义了输入数据x经过本模块后的输出计算过程
    def forward(self, x):
        # 将输入数据x传递给block中的各个层进行处理
        x = self.block(x)
        # 返回处理后的结果
        return x


# 定义一个分类器类，用于构建完整的神经网络模型
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        # 继承自nn.Module类，并初始化父类
        super(Classifier, self).__init__()

        # 构建神经网络的全连接部分，使用Sequential容器组织各个层
        # 首先是一个BasicBlock，用于从输入层转换到隐藏层
        # 然后根据hidden_layers的数值，重复添加hidden_dim到hidden_dim的BasicBlock
        # 最后，添加一个线性层，将最后一个隐藏层映射到输出层，输出维度为output_dim
        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),  # 第一个基础模块，开始转换输入
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],  # 重复的基础模块，构成多层网络
            nn.Linear(hidden_dim, output_dim)  # 输出层，决定模型的分类种类数
        )

    # 前向传播方法，定义模型如何处理输入数据x并产生输出
    def forward(self, x):
        # 将输入数据x传递给整个神经网络模型（self.fc），得到最终的输出
        x = self.fc(x)
        # 返回模型的预测输出
        return x

# Hyper-parameters

In [ ]:
# 数据参数
# 待办事项：为中等基线模型修改 "concat_nframes" 的值
concat_nframes = 3   # 要拼接的帧数，n 必须为奇数（总帧数为 2k+1 = n）
train_ratio = 0.75   # 用于训练的数据比例，剩余部分用于验证

# 训练参数
seed = 1213          # 随机种子
batch_size = 512        # 批大小
num_epoch = 10         # 训练轮次的数量
learning_rate = 1e-4      # 学习率
model_path = './model.ckpt'  # 模型检查点保存的路径

# 模型参数
# 待办事项：为中等基线模型修改 "hidden_layers" 或 "hidden_dim" 的值
input_dim = 39 * concat_nframes  # 模型的输入维度，不应更改此值
hidden_layers = 2          # 隐藏层的数量
hidden_dim = 64           # 隐藏层的维度

# Dataloader

In [ ]:
# 导入 DataLoader 类，用于创建数据加载器，帮助我们管理和加载数据
from torch.utils.data import DataLoader
# 导入垃圾回收模块，用于手动管理内存
import gc
# 导入PyTorch库中的torch模块，包含基本的Tensor操作和神经网络构建功能
import torch

# 调用自定义的函数same_seeds来设置随机种子，确保每次运行时随机操作的结果都相同，便于复现实验结果
same_seeds(seed)

# 检测当前系统是否有GPU可用，如果有，则将计算任务分配给GPU，提高计算速度；否则，使用CPU进行计算
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 打印当前使用的计算设备信息
print(f'当前使用的设备为: {device}')

# 预处理数据，准备训练集
# 调用preprocess_data函数处理特征文件，该函数会读取音频特征并根据concat_nframes拼接帧，同时根据train_ratio划分训练集，
# 并使用random_seed确保数据划分的一致性
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone',
                                   concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
# 同样的预处理步骤应用于验证集数据
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone',
                               concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# 使用处理好的数据创建数据集对象
# LibriDataset是一个自定义的数据集类，它继承自torch.utils.data.Dataset，负责组织数据和标签对
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# 清理原始特征变量以释放内存，避免在后续训练过程中占用过多资源
# 使用gc.collect()手动触发Python的垃圾回收机制
del train_X, train_y, val_X, val_y
gc.collect()

# 创建数据加载器，这是训练和验证过程中用来迭代数据的关键组件
# 训练集的数据加载器使用shuffle=True来打乱数据顺序，有助于模型训练时的泛化能力
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# 验证集的数据加载器则通常不打乱数据顺序，shuffle=False
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Training

In [ ]:
# 深度学习模型训练的标准流程，包括模型的训练和验证过程，以及模型性能的监控和最优模型的保存策略

# 实例化模型，并将其移动到之前确定的设备（CPU或GPU）
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

# 定义损失函数为交叉熵损失，适用于多分类问题
criterion = nn.CrossEntropyLoss()

# 初始化Adam优化器，用于更新模型参数。学习率设置为之前定义的learning_rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 初始化最佳验证精度变量，用于跟踪模型的最佳性能
best_acc = 0.0

# 主训练循环，遍历num_epoch个周期
for epoch in range(num_epoch):
    # 初始化训练准确率和损失，以及验证准确率和损失
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # 开始训练阶段
    model.train()  # 设置模型为训练模式，启用dropout等特性
    for batch in tqdm(train_loader):  # 使用tqdm显示进度条，增强用户体验
        features, labels = batch  # 从数据加载器中获取特征和标签
        features, labels = features.to(device), labels.to(device)  # 移动数据到指定设备

        # 梯度清零，防止梯度累加
        optimizer.zero_grad()

        # 前向传播，得到模型输出
        outputs = model(features)

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播计算梯度
        loss.backward()

        # 更新模型参数
        optimizer.step()

        # 计算训练准确率
        _, train_pred = torch.max(outputs, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # 开始验证阶段，模型切换到评估模式，关闭dropout等
    model.eval()
    with torch.no_grad():  # 不进行梯度计算，节省内存且速度更快
        for batch in tqdm(val_loader):
            features, labels = batch
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            # 计算验证损失
            loss = criterion(outputs, labels)

            # 计算验证准确率
            _, val_pred = torch.max(outputs, 1)
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
            val_loss += loss.item()

    # 输出当前轮次的训练和验证指标
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    train_acc_rate = train_acc / len(train_set)
    val_acc_rate = val_acc / len(val_set)
    print(f'Epoch [{epoch+1:03d}/{num_epoch:03d}], Train Acc: {train_acc_rate:.5f}, Loss: {avg_train_loss:.5f} | Val Acc: {val_acc_rate:.5f}, Loss: {avg_val_loss:.5f}')

    # 如果当前模型在验证集上的表现优于之前记录的最佳模型，则保存当前模型参数
    if val_acc_rate > best_acc:
        best_acc = val_acc_rate
        torch.save(model.state_dict(), model_path)  # state_dict保存模型参数
        print(f'Saved best model with accuracy: {best_acc:.5f}')

In [ ]:
# 删除训练集和验证集的数据集对象
# 当模型训练完成后，这些数据集不再需要，因此可以释放它们占用的内存
del train_set, val_set

# 同样删除数据加载器对象，进一步释放内存资源
del train_loader, val_loader

# 手动触发Python的垃圾回收机制来回收刚刚删除的对象所占用的内存空间
# 这对于内存管理尤为重要，尤其是在处理大规模数据集时
gc.collect()

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# 加载测试数据
# 对测试集进行预处理，与之前处理训练集和验证集的步骤相似，但测试集没有对应的标签（设为None）
# 因为在测试阶段我们通常只关心模型的预测结果而非训练或调整模型
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)

# 使用处理后的测试特征创建数据集对象
# 注意，这里第二个参数传入None，表示测试集中没有对应的标签数据（或在评估时不需要使用）
test_set = LibriDataset(test_X, None)

# 创建测试数据的 DataLoader，用于在测试模型时批量加载数据
# 测试时通常不需打乱数据顺序，因此 shuffle 设为 False
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
# 初始化模型
# 根据给定的输入维度、隐藏层结构和隐藏层维度创建分类器模型实例
# 并将模型转移到预先设定的设备上（如GPU），以利用硬件加速
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

# 加载已训练好的模型权重
# 使用torch.load函数从指定路径加载之前保存的模型状态字典
# 这一步骤使得我们可以继续使用之前训练好的模型进行推断或者进一步训练
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
# 初始化预测结果数组，用于存储模型对测试集每个样本的预测类别
pred = np.array([], dtype=np.int32)

# 将模型设置为评估模式，这会关闭诸如Dropout这样的训练时特有的层，确保预测的一致性和稳定性
model.eval()

# 使用with语句上下文管理器，确保在预测过程中不进行梯度计算，减少内存消耗并加速预测过程
with torch.no_grad():
    # 遍历测试数据加载器中的每个批次
    for i, batch in enumerate(tqdm(test_loader)):
        # 从批次中提取特征，并将它们移动到计算设备上
        features = batch
        features = features.to(device)

        # 对批次数据进行前向传播，得到模型的输出
        outputs = model(features)

        # 获取每一样本预测概率最高的类别索引
        # torch.max返回两个值，第一个是最大值，第二个是最大值的索引（即预测类别）
        _, test_pred_batch = torch.max(outputs, 1)

        # 将当前批次的预测结果从Tensor转换为NumPy数组，并拼接到总的预测结果数组中
        # 这里使用np.concatenate函数沿着指定轴（axis=0，即行方向）拼接数组
        pred = np.concatenate((pred, test_pred_batch.cpu().numpy()), axis=0)

Write prediction to a CSV file.

After finish running this block, prediction.csv 文件将包含每个测试样本的预测类别，便于后续分析或提交到相应的评测平台

In [ ]:
# 使用with语句打开一个文件，命名为'prediction.csv'，以写入模式('w')打开
# 这样做可以确保文件操作安全，即操作完成后自动关闭文件
with open('prediction.csv', 'w') as f:
    # 写入CSV文件的头部信息，包括两列：Id和Class
    f.write('Id,Class\n')

    # 遍历预测结果列表(pred)的索引i和对应的预测值y
    for i, y in enumerate(pred):
        # 将每个样本的索引i和其预测类别y格式化写入文件
        # 使用逗号分隔Id和Class，每完成一个样本的写入后换行
        f.write('{},{}\n'.format(i, y))